In [9]:
import estnltk
import pandas as pd
from collections import Counter
from estnltk.taggers.adjective_phrase_tagger.adj_phrase_tagger import AdjectivePhraseTagger
import re

In [5]:
data = pd.read_csv("../../data/meediadata.csv")

In [6]:
def read_keywords(filename):
    keywords = []
    with open(filename) as infile:
        for line in infile:
            keywords.append(line.strip())
            
    return keywords

keywords = read_keywords("ohtuleht_keywords.txt")
fotod_videod = read_keywords("fotod.txt")

In [10]:
def split_sentence(sentence):
    pattern = "(" + "|".join([k for k in keywords if "|" not in k]) + ")" + "([A-Z]|[0-9])"
    res = re.search(pattern, sentence)
    if res is None:
        return([sentence.strip()])
    else:
        return(split_sentence(sentence[:res.start()]) + split_sentence(sentence[res.end() - 1:]))

def clean_data(sentence, allikas):
    sentence = sentence.strip().replace("«", "")
    if allikas != "ohtuleht":
        return [sentence]
    
    res = [(m.start(0), m.end(0)) for m in re.finditer("\([0-9]+\)", sentence)]

    sentences = split_sentence(sentence)
    
    if len(res) == 1 and res[0][1] != len(sentence):
        res = [res[0], [len(sentence), len(sentence)]]
        
    
    if len(res) > 1:
        sentences = []
        start = 0
        for c in res:
            temp = sentence[start:c[1]]
            temp_sentences = split_sentence(temp)
            
            for s in temp_sentences:
                sentences.append(s)   
            start = c[1]
    
    
    for j in range(len(sentences)):
        sentences[j] = sentences[j].strip()
        if sentences[j] == "":
            continue
        parts = sentences[j].strip().split()
        if len(parts) > 1 and len(parts[-1]) >= 3 and parts[-1][0] == "(" and parts[-1][-1] == ")" and parts[-1][1:-1].isdigit():
            sentences[j] = " ".join(parts[:-1])
        parts = sentences[j].strip().split()
        upper = [i for i in range(len(parts[0])) if parts[0][i].isupper()]
        if len(upper) >= 2 and min(upper[1:]) >= 1 and parts[0][0].isalpha() and upper[1] - upper[0] > 1:
            if "-" in parts[0] and parts[0].index("-") < min(upper[1:]):
                pass
            else:
                parts[0] = parts[0][min(upper[1:]):]
                sentences[j] = " ".join(parts).strip()
        for k in keywords:
            if sentences[j].startswith(k) and len(sentences[j]) > len(k) and (sentences[j][len(k)].isupper() or sentences[j][len(k)].isdigit()):
                sentences[j] = sentences[j][len(k):].strip()

    return [s for s in sentences if len(s) > 0]

def remove_foto_video(sentence):
    current_k = ""
    for k in fotod_videod:
        if sentence.startswith(k) and len(sentence) > len(k) and len(k) > len(current_k):
            current_k = k
    return sentence.lstrip(current_k)

In [11]:
pealkirjad = []
allikad = []
ajad = []
cleaned_pealkirjad = []
altered_pealkirjad = []
for row in data.iterrows():
    row = row[1]
    cleaned = clean_data(row["pealkiri"], row["allikas"])
    if len(cleaned) > 1:
        pass
        #print(cleaned)
    for sentence in cleaned:
        pealkirjad.append(row["pealkiri"])
        allikad.append(row["allikas"])
        ajad.append(row["aeg"])
        cleaned_pealkirjad.append(sentence)
        altered_pealkirjad.append(remove_foto_video(sentence))

In [12]:
data = pd.DataFrame({"pealkiri": pealkirjad, "allikas": allikad, "aeg": ajad, "cleaned_pealkiri": cleaned_pealkirjad,
                    "altered_pealkiri": altered_pealkirjad})
print(data.shape)
data.head()

(8979, 5)


,aeg,allikas,altered_pealkiri,cleaned_pealkiri,pealkiri
0,2017-04-12 15:06:49,postimees,Küberpättidel õnnestus Eestist tundlikku infot...,Küberpättidel õnnestus Eestist tundlikku infot...,Küberpättidel õnnestus Eestist tundlikku infot...
1,2017-04-12 15:06:49,postimees,Trumpi poeg: isa otsust rünnata Süüriat mõjuta...,Trumpi poeg: isa otsust rünnata Süüriat mõjuta...,Trumpi poeg: isa otsust rünnata Süüriat mõjuta...
2,2017-04-12 15:06:49,postimees,Stockholmi ründaja polegi usbekk Rakhmat Akilov?,Stockholmi ründaja polegi usbekk Rakhmat Akilov?,Stockholmi ründaja polegi usbekk Rakhmat Akilov?
3,2017-04-12 15:06:49,postimees,Vanasadamas ristiti viimaks ka parvlaev Piret,Fotod: Vanasadamas ristiti viimaks ka parvlaev...,Fotod: Vanasadamas ristiti viimaks ka parvlaev...
4,2017-04-12 15:06:49,postimees,"Lauri uutest maksudest: miks proovida asju, mi...","Lauri uutest maksudest: miks proovida asju, mi...","Lauri uutest maksudest: miks proovida asju, mi..."


In [13]:
data2 = data.drop_duplicates(subset = ["cleaned_pealkiri", "allikas"])
print(data2.shape)
df_all = pd.DataFrame(columns=['word_texts', 'lemmas', 'postags', 'postag_descriptions', 'id',
       'pealkiri', 'allikas', 'aeg', 'cleaned_pealkiri', 'altered_pealkiri'])

i = 1
for row in data2.iterrows():
    t = row[1]["altered_pealkiri"]
    df = estnltk.Text(t).get.word_texts.lemmas.postags.postag_descriptions.as_dataframe
    df["id"] = i
    df["pealkiri"] = row[1]["pealkiri"]
    df["allikas"] = row[1]["allikas"]
    df["aeg"] = row[1]["aeg"]
    df["cleaned_pealkiri"] = row[1]["cleaned_pealkiri"]
    df["altered_pealkiri"] = row[1]["altered_pealkiri"]
    df_all = pd.concat([df_all, df])
    i += 1
    if i % 500 == 0:
        print(i)
    


(6352, 5)
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000


In [16]:
def split_word(word):
    if "|" in word:
        parts = word.split("|")
        if len(parts[0]) > 0:
            return parts[0]
    return word

df_all["lemmas_clean"] = df_all["lemmas"].apply(split_word)

In [380]:
df_all["id"] = df_all["id"].astype("int")
df_all.head()
df_all.to_csv("meedia_data.csv")

In [296]:
import re

pattern = "(" + "|".join(keywords) + ")" + "([A-Z]|[0-9])"
res = re.search(pattern, "kes sa oled Elu fkd asd fk")
res


In [226]:
a = 'Ilm ja loodusSAADA FOTO! Käes aeg ilusaid lumiseid piltide teha. JÄLLE!\xa0(6)Saada meile lumiseid fotosid aadressile ohtuleht@ohtuleht.ee TallinnOOTAMATU LUMESADU: lennud Tallinnast Helsingisse ja Riiga on halva ilma tõttu tühistatud JalgpallSTATISTIKAPOMM | Väravamasin Ronaldo, anonüümne Vardy ja Prantsusmaa legendidega ühele pulgale tõusnud Griezmann NBASTATISTIKAPOMM | Täpne Jordan, võimsa rünnakuga Warriors ja kolmikduublimasin Westbrook'
clean_data(a, "ohtuleht")

['Ilm ja loodusSAADA FOTO! Käes aeg ilusaid lumiseid piltide teha. JÄLLE!\xa0(6)Saada meile lumiseid fotosid aadressile ohtuleht@ohtuleht.ee TallinnOOTAMATU LUMESADU: lennud Tallinnast Helsingisse ja Riiga on halva ilma tõttu tühistatud JalgpallSTATISTIKAPOMM | Väravamasin Ronaldo, anonüümne Vardy ja Prantsusmaa legendidega ühele pulgale tõusnud Griezmann NBASTATISTIKAPOMM | Täpne Jordan, võimsa rünnakuga Warriors ja kolmikduublimasin Westbrook']

In [241]:
#keywords

In [156]:
data[data["pealkiri"].str.strip() != data["cleaned_pealkiri"]]["allikas"].value_counts()
#data[(data["pealkiri"].str.strip() != data["cleaned_pealkiri"]) & (data["allikas"] == "postimees")]
for i in range(1200,1300):
    print(data[data["allikas"] == "ohtuleht"].iloc[i]["pealkiri"])
    print(data[data["allikas"] == "ohtuleht"].iloc[i]["cleaned_pealkiri"])

EDASI, CARDIFISSE! Leicester City Meistrite liiga poolfinaalidesse – (hele)sinine unistus, aga šanss on olemas!
EDASI, CARDIFISSE! Leicester City Meistrite liiga poolfinaalidesse – (hele)sinine unistus, aga šanss on olemas!
MARGUS VAHER JULGUSTAB NAISI MAGAMISTOAS HULLAMA: mees ei vaja voodis ideaalmõõtudega bimbot, vaid naist, kes laseb end vabaks (59)
MARGUS VAHER JULGUSTAB NAISI MAGAMISTOAS HULLAMA: mees ei vaja voodis ideaalmõõtudega bimbot, vaid naist, kes laseb end vabaks
Kylie Minogue semmib prints Andrew'ga?!
Kylie Minogue semmib prints Andrew'ga?!
Birgit Sarrap näitas võrratut figuuri!
Birgit Sarrap näitas võrratut figuuri!
Anu Saagim: lahkusin Postimehest, sest ma ei olnud seal enam õnnelik! (74)
Anu Saagim: lahkusin Postimehest, sest ma ei olnud seal enam õnnelik!
TarbijaFOTOD | "Põnev" leid - soodusjäätise sees olid kiletükid (4)Tervist24MARATONIHOOAEG ON KÄES: kuidas võistlusteks tervisesäästlikult valmistuda? ReisReisiohutuse kolm võtmeküsimust Ilu ja moodKEVAD ON FLIRTIM

In [5]:
def sonad(data, ajakiri, sonatyyp):
    laused = list(data[data["allikas"] == ajakiri]["pealkiri"].drop_duplicates())
    dfs = [estnltk.Text(l).get.word_texts.lemmas.postags.postag_descriptions.as_dataframe for l in laused]
    sonad = pd.concat(dfs)
    sonad_lemmad = sonad[["lemmas", "postags", "postag_descriptions"]]
    sonad_count = pd.DataFrame(sonad_lemmad.groupby(["lemmas", "postags", "postag_descriptions"]).size()).reset_index()
    sonad_count.columns = ["sona", "tag", "tag_desc", "count"]
    sonad_count = sonad_count.sort_values("count", ascending=False)
    return sonad_count[sonad_count["tag_desc"].isin(sonatyyp)]

In [16]:
omadus = ["omadussõna algvõrre", "omadussõna keskvõrre", "omadussõna ülivõrre"]
omadus = ["omadussõna ülivõrre"]
sonad(data, "postimees", omadus).head()

,sona,tag,tag_desc,count
2226,suurim,U,omadussõna ülivõrre,11
775,enim,U,omadussõna ülivõrre,2
1784,parim,U,omadussõna ülivõrre,2
1590,moodsaim,U,omadussõna ülivõrre,1
1514,mainekaim,U,omadussõna ülivõrre,1


In [17]:
sonad(data, "delfi", omadus).head()

,sona,tag,tag_desc,count
2818,parim,U,omadussõna ülivõrre,9
3456,suurim,U,omadussõna ülivõrre,7
1324,enim,U,omadussõna ülivõrre,2
3994,vanim,U,omadussõna ülivõrre,2
2882,pikim,U,omadussõna ülivõrre,2


In [18]:
sonad(data, "ohtuleht", omadus).head()

,sona,tag,tag_desc,count
2291,parim,U,omadussõna ülivõrre,14
2768,suurim,U,omadussõna ülivõrre,4
3130,vanim,U,omadussõna ülivõrre,3
2082,moodsaim,U,omadussõna ülivõrre,3
601,Meeledparim,U,omadussõna ülivõrre,2


In [19]:
sonad(data, "err", omadus).head()

,sona,tag,tag_desc,count
1582,vanim,U,omadussõna ülivõrre,2
1189,populaarseim,U,omadussõna ülivõrre,1
1694,ülim,U,omadussõna ülivõrre,1
1136,parim,U,omadussõna ülivõrre,1
1540,vaadatuim,U,omadussõna ülivõrre,1


In [104]:
estnltk.Text(laused[1]).get.word_texts.lemmas.postags.postag_descriptions.as_dataframe

,word_texts,lemmas,postags,postag_descriptions
0,Trumpi,Trump|Trumpi,H,pärisnimi
1,poeg,poeg,S,nimisõna
2,:,:,Z,lausemärk
3,isa,isa,S,nimisõna
4,otsust,otsus,S,nimisõna
5,rünnata,ründama,V,tegusõna
6,Süüriat,Süüria,H,pärisnimi
7,mõjutas,mõjutama,V,tegusõna
8,Ivanka,Ivanka|Ivankas,H,pärisnimi


In [27]:
data[data["allikas"] == "ohtuleht"]

,Unnamed: 0,pealkiri,allikas,aeg
1949,1950,PROPAGANDA MISSUGUNE: Venemaa saatis Sinimäged...,ohtuleht,2017-04-12 15:06:49
1950,1951,"VANDENÕUTEOREETIK: Estonia uputati, et saada l...",ohtuleht,2017-04-12 15:06:49
1951,1952,RETRODRIFT | Kalvi-Kalle oli abieluks valmis j...,ohtuleht,2017-04-12 15:06:49
1952,1953,FANTASTILINE! Anett Kontaveit sai maailma 78. ...,ohtuleht,2017-04-12 15:06:49
1953,1954,ÕHTULEHE EKSPERIMENT | Kahe kuuga pani kõhu pe...,ohtuleht,2017-04-12 15:06:49
1954,1955,Putin: Venemaa ja USA suhted lähevad aina halv...,ohtuleht,2017-04-12 15:06:49
1955,1956,SAKSLASED PAHANDAVAD: kuidas peaks Dortmundi B...,ohtuleht,2017-04-12 15:06:49
1956,1957,"KEVADINE DETOX: 11 nõksu, kuidas oma keha igap...",ohtuleht,2017-04-12 15:06:49
1957,1958,"""PROOVIABIELU"" JÕUAB TÄNA ÕHTUL FINAALINI: kel...",ohtuleht,2017-04-12 15:06:49
1958,1959,"KGB ""hamster"" kogus Eesti kultuurieliidi eraki...",ohtuleht,2017-04-12 15:06:49


In [29]:
tagger = AdjectivePhraseTagger(return_layer=True)

In [59]:
sent = estnltk.Text("Tere normaalne kes sä oled?")
tagger.tag(sent)

BrokenPipeError: [Errno 32] Broken pipe